In [1]:
from selenium import webdriver
import selenium
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [2]:
driver = webdriver.Chrome(r'C:\\Users\nedas\anaconda3\pkgs\chromedriver_win32\chromedriver.exe')
driver.set_page_load_timeout(15)
driver.get('https://sellercentral.amazon.com/forums/c/account-health')

for i in range(30):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)

data = driver.find_elements_by_css_selector('a[class = "title raw-link raw-topic-link"]')
titles = [x.text for x in data]
links = [x.get_attribute('href') for x in data]

data_subcat = driver.find_elements_by_css_selector('span[class = "category-name"]')
sub_cats = [x.text for x in data_subcat[2:]]

contents = []
for link in links:
    driver.get(link)
    data = driver.find_elements_by_css_selector('div[class = "cooked"]')
    content_tempo = [x.text.replace('\n', ' ') for x in data]
    contents.append(content_tempo)

time.sleep(5)
driver.quit()

In [3]:
posts = []
responses = []
for line in contents:
    posts.append(line[0])
    responses.append(line[1:])

In [4]:
def create_response_df(urls, resps):
    column_names = ['URL', 'Response Text']
    df_resps = pd.DataFrame(columns = column_names)
    for url,line in zip(urls, resps):
        n = len(line)
        sub_urls = [url] * n
        new_df = pd.DataFrame(list(zip(sub_urls, line)), columns = column_names)
        df_resps = pd.concat([df_resps, new_df])
    return df_resps

In [22]:
DF1 = pd.DataFrame(titles, columns=['Title'])
DF1['Category'] = 'Account Health'
DF1['Sub-category'] = sub_cats
DF1['Post Text'] = posts
DF1['URL'] = links

In [34]:
DF1.head(50)

,Title,Category,Sub-category,Post Text,URL
0,Account Suspended After 5 years out of nowhere...,Account Health,Related Accounts,Our brand was suspended about 25 minutes ago. ...,https://sellercentral.amazon.com/forums/t/acco...
1,Another related account suspension,Account Health,Related Accounts,"Hi guys, my first post here (long, apologies) ...",https://sellercentral.amazon.com/forums/t/anot...
2,Used Sold As New - Account Review,Account Health,Account Health,Ive reviewed a lot of discussion in this forum...,https://sellercentral.amazon.com/forums/t/used...
3,Section 3 - related to another account,Account Health,Related Accounts,"Hello, I am very new at this and I cant unders...",https://sellercentral.amazon.com/forums/t/sect...
4,"Amazon Shut Down Our Familys Business, and We ...",Account Health,Related Accounts,A tough lesson on partnering with the most pow...,https://sellercentral.amazon.com/forums/t/amaz...
5,"Used Sold As New Plan of Action, FBM, Brand Owner",Account Health,Product Condition,Hello there! Weve received the dreaded Listing...,https://sellercentral.amazon.com/forums/t/used...
6,Account Suspended-Utility Bill,Account Health,Account Health,So Amazon suspended the account soon after cre...,https://sellercentral.amazon.com/forums/t/acco...
7,Rejected trademark report,Account Health,Intellectual Property,I have found a seller that is selling products...,https://sellercentral.amazon.com/forums/t/reje...
8,Related Account Suspension NEED HELP,Account Health,Related Accounts,"Hello Amazon sellers, I have recently had my s...",https://sellercentral.amazon.com/forums/t/rela...
9,Please restore our seller account or refund to...,Account Health,Account Health,Our seller account was suspended by amazon bec...,https://sellercentral.amazon.com/forums/t/plea...


In [24]:
DF2 = create_response_df(links, responses)

In [35]:
DF2.head(50)

,URL,Response Text
0,https://sellercentral.amazon.com/forums/t/acco...,Do you have any other accounts using your addr...
1,https://sellercentral.amazon.com/forums/t/acco...,Have the same issue here. Account was deactiva...
2,https://sellercentral.amazon.com/forums/t/acco...,We just got suspended too for the same reason....
3,https://sellercentral.amazon.com/forums/t/acco...,"Not at all, theres only 1 Amazon account."
4,https://sellercentral.amazon.com/forums/t/acco...,"Thats exactly our problem, how the hell are we..."
5,https://sellercentral.amazon.com/forums/t/acco...,I just appealed mine as an error and sent in a...
6,https://sellercentral.amazon.com/forums/t/acco...,This may not apply to anyone but remember not ...
7,https://sellercentral.amazon.com/forums/t/acco...,I appealed as an error also. Its probably a pr...
8,https://sellercentral.amazon.com/forums/t/acco...,The same issue. Never had any problems or seri...
9,https://sellercentral.amazon.com/forums/t/acco...,I have no idea if this is related: today I not...


In [26]:
#removes non english characters from titles and content
DF1['Title'] = DF1['Title'].str.replace(r'[^\x00-\x7F]+', '')
DF1['Post Text'] = DF1['Post Text'].str.replace(r'[^\x00-\x7F]+', '')

DF2['Response Text'] = DF2['Response Text'].str.replace(r'[^\x00-\x7F]+', '')

In [36]:
pd.set_option('display.max_rows', None)
DF1.head(50)

,Title,Category,Sub-category,Post Text,URL
0,Account Suspended After 5 years out of nowhere...,Account Health,Related Accounts,Our brand was suspended about 25 minutes ago. ...,https://sellercentral.amazon.com/forums/t/acco...
1,Another related account suspension,Account Health,Related Accounts,"Hi guys, my first post here (long, apologies) ...",https://sellercentral.amazon.com/forums/t/anot...
2,Used Sold As New - Account Review,Account Health,Account Health,Ive reviewed a lot of discussion in this forum...,https://sellercentral.amazon.com/forums/t/used...
3,Section 3 - related to another account,Account Health,Related Accounts,"Hello, I am very new at this and I cant unders...",https://sellercentral.amazon.com/forums/t/sect...
4,"Amazon Shut Down Our Familys Business, and We ...",Account Health,Related Accounts,A tough lesson on partnering with the most pow...,https://sellercentral.amazon.com/forums/t/amaz...
5,"Used Sold As New Plan of Action, FBM, Brand Owner",Account Health,Product Condition,Hello there! Weve received the dreaded Listing...,https://sellercentral.amazon.com/forums/t/used...
6,Account Suspended-Utility Bill,Account Health,Account Health,So Amazon suspended the account soon after cre...,https://sellercentral.amazon.com/forums/t/acco...
7,Rejected trademark report,Account Health,Intellectual Property,I have found a seller that is selling products...,https://sellercentral.amazon.com/forums/t/reje...
8,Related Account Suspension NEED HELP,Account Health,Related Accounts,"Hello Amazon sellers, I have recently had my s...",https://sellercentral.amazon.com/forums/t/rela...
9,Please restore our seller account or refund to...,Account Health,Account Health,Our seller account was suspended by amazon bec...,https://sellercentral.amazon.com/forums/t/plea...


In [37]:
DF2.head(50)

,URL,Response Text
0,https://sellercentral.amazon.com/forums/t/acco...,Do you have any other accounts using your addr...
1,https://sellercentral.amazon.com/forums/t/acco...,Have the same issue here. Account was deactiva...
2,https://sellercentral.amazon.com/forums/t/acco...,We just got suspended too for the same reason....
3,https://sellercentral.amazon.com/forums/t/acco...,"Not at all, theres only 1 Amazon account."
4,https://sellercentral.amazon.com/forums/t/acco...,"Thats exactly our problem, how the hell are we..."
5,https://sellercentral.amazon.com/forums/t/acco...,I just appealed mine as an error and sent in a...
6,https://sellercentral.amazon.com/forums/t/acco...,This may not apply to anyone but remember not ...
7,https://sellercentral.amazon.com/forums/t/acco...,I appealed as an error also. Its probably a pr...
8,https://sellercentral.amazon.com/forums/t/acco...,The same issue. Never had any problems or seri...
9,https://sellercentral.amazon.com/forums/t/acco...,I have no idea if this is related: today I not...


In [30]:
len(DF2.URL.unique())

868

In [31]:
len(DF1.URL.unique())

930

#### Note:
Checking the number of unique urls above, it seems that some posts don't have replies. This should be taken into account when merging again. Should do a left or right merge instead of an inner merge to avoid losing data.

In [32]:
DF1.to_csv('DF1.csv', index = False)
DF2.to_csv('DF2.csv', index = False)